In [1]:
INPUT_VERSION = 15
OUTPUT_VERSION = 3

S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/test01/output/"

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/test01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017test01/output"

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V%0.2d.pkl" %(INPUT_VERSION)
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M09D15_RH_Add_Basin_Data_V%0.2d" %(OUTPUT_VERSION)

In [2]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [3]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [4]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V14.pkl to ../../../../data/test01/input/mergedZonalStatsEE_V14.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V15.pkl to ../../../../data/test01/input/mergedZonalStatsEE_V15.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V14.csv to ../../../../data/test01/input/mergedZonalStatsEE_V14.csv
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V15.csv to ../../../../data/test01/input/mergedZonalStatsEE_V15.csv


In [5]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/test01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [6]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [7]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)

In [8]:
df_ee = pd.read_pickle(inputLocationEE)

In [9]:
df_ee.index.names = ['PFAF_ID']

In [10]:
df_ee.head()

,count_Hybas06,mean_Hybas06,count_area_30s_m2,mean_area_30s_m2,total_area_30s_m2,count_PIndWW_monthY2014M11,mean_PIndWW_monthY2014M11,total_volume_PIndWW_monthY2014M11,count_PLivWW_monthY2014M07,mean_PLivWW_monthY2014M07,...,total_volume_IrrLinearWN_monthY2014M01,count_PDomWW_monthY2014M10,mean_PDomWW_monthY2014M10,total_volume_PDomWW_monthY2014M10,count_PIrrWW_monthY2014M04,mean_PIrrWW_monthY2014M04,total_volume_PIrrWW_monthY2014M04,count_IrrLinearWW_monthY2014M12,mean_IrrLinearWW_monthY2014M12,total_volume_IrrLinearWW_monthY2014M12
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,2536,111011.0,2536,743658.186761,1.885917e+09,2536,0.002146,4.048026e+06,2536,1.389657e-07,...,55779.511103,2536,0.000501,944356.094213,2536,0.000158,298467.591262,2536,0.000066,124430.115923
111012,3921,111012.0,3921,746186.462653,2.925797e+09,3921,0.000229,6.688613e+05,3921,6.733557e-07,...,0.000000,3921,0.000005,13213.019966,3921,0.000000,0.000000,3921,0.000000,0.000000
111013,1194,111013.0,1194,747422.836265,8.924229e+08,1194,0.000694,6.191623e+05,1194,5.213650e-08,...,0.000000,1194,0.000013,11774.732797,1194,0.000000,0.000000,1194,0.000000,0.000000
111014,5605,111014.0,5605,750449.270450,4.206268e+09,5605,0.000018,7.558842e+04,5605,2.151802e-07,...,0.000000,5605,0.000003,10953.398148,5605,0.000000,0.000000,5605,0.000000,0.000000
111015,21873,111015.0,21873,758792.279231,1.659706e+10,21769,0.000541,8.983459e+06,21769,2.745748e-08,...,0.000000,21769,0.000021,340838.576971,21769,0.000000,0.000000,21769,0.000000,0.000000


In [11]:
dfTest = df_ee[df_ee["total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] < 0 ]

In [17]:
mask = df_ee["total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] < 0

df_ee.loc[mask, "total_volume_reducedmeanrunoff_month_Y1960Y2014M01"] = 0

In [19]:
df_ee.total_volume_reducedmeanrunoff_month_Y1960Y2014M01.min()

0.0